In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
from sklearn.cluster import KMeans
from google.colab import files
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor

class RouterOptimizer:
    def __init__(self):
        self.rssi_threshold = -70 #RSSI mínimo a ser considerado como cobertura
        self.tx_power = 23  # dBm || Potência do roteador
        self.freq_mhz = 2400  # MHz || Frequência do WiFI
        self.scale_factor = 2  # Escala de visualização do gráfico
        self.distance_conversion = 0.5  # Fator de conversão de unidades para metros (0.5m por unidade)

        self.weight_mapping = {
            'blue': 16.67,    # Parede (concreto)
            'red': 7,         # Janela
            'green': 6.81,    # Porta
            'yellow': 4,      # MDF
            'default': 1      # Passagem livre
        }

    def load_graph(self):
        """Carrega o grafo do usuário"""
        uploaded = files.upload()
        graph_file = next(iter(uploaded))
        G = nx.read_graphml(graph_file)
        return nx.relabel_nodes(G, {n: eval(n) for n in G.nodes()})

    def show_graph(self, G, title="Grafo de Navegação"):
        """Exibe o grafo com matplotlib"""
        scale_factor = self.scale_factor
        pos = {n: (n[0] * scale_factor, n[1] * scale_factor) for n in G.nodes()}
        plt.figure(figsize=(16, 12))

        # Desenhar arestas
        edge_colors = [self.weight_colors.get(G[u][v].get('weight', 1), 'black') 
                      for u, v in G.edges()]
        nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=1.2, alpha=0.6)

        # Desenhar nós
        nx.draw_networkx_nodes(
            G, pos, node_color='black',
            node_size=20, edgecolors='white', linewidths=0.5
        )

        plt.title(title)
        plt.axis('equal')
        plt.axis('off')
        plt.show()

    def calc_fspl(self, distance_m):
        """Calcula a perda de percurso no espaço livre"""
        if distance_m == 0:
            return 0
        distance_km = distance_m / 1000.0
        return 20 * np.log10(distance_km) + 20 * np.log10(self.freq_mhz) + 32.44

    @lru_cache(maxsize=None)
    def get_path_and_loss(self, G, source, target):
        """Calcula o caminho e a perda por obstáculos entre dois nós"""
        try:
            path = nx.shortest_path(G, source=source, target=target, weight='weight')
            obstacle_loss = sum(G[p][q]['weight'] for p, q in zip(path[:-1], path[1:]))
            return path, obstacle_loss
        except (nx.NetworkXNoPath, KeyError):
            return None, float('inf')

    def compute_rssi_for_node(self, G, node, routers):
        """Calcula o melhor RSSI para um nó em relação aos roteadores"""
        best_rssi = -100.0
        for router in routers:
            if node == router:
                continue  # Ignora o próprio roteador
            path, obstacle_loss = self.get_path_and_loss(G, node, router)
            if path is None:
                continue
            euclidean_dist = np.hypot(node[0] - router[0], node[1] - router[1])
            fspl = self.calc_fspl(euclidean_dist * self.distance_conversion)
            rssi = self.tx_power - fspl - obstacle_loss
            if rssi > best_rssi:
                best_rssi = rssi
        return best_rssi

    def evaluate_coverage(self, G, routers):
        """Avalia a cobertura e RSSI médio para uma configuração de roteadores"""
        node_list = list(G.nodes())
        with ThreadPoolExecutor() as executor:
            rssi_values = list(executor.map(
                lambda node: self.compute_rssi_for_node(G, node, routers), 
                node_list
            ))
        rssi_values = np.array(rssi_values)
        coverage = np.sum(rssi_values >= self.rssi_threshold) / len(rssi_values) * 100
        valid_rssi = rssi_values[rssi_values > -100]
        avg_rssi = np.mean(valid_rssi) if len(valid_rssi) > 0 else -100
        return coverage, avg_rssi, rssi_values

    def router_distance_penalty(self, routers):
        """Calcula penalização por roteadores muito próximos"""
        total = 0
        for a, b in combinations(routers, 2):
            d = np.hypot(a[0] - b[0], a[1] - b[1])
            total += 1 / (d + 1e-3)
        return total

    def find_best_routers(self, G, num_roteadores, max_iter=50):
        """Encontra a melhor posição para os roteadores"""
        nodes = list(G.nodes())
        best = {'coverage': 0, 'avg_rssi': -100, 'routers': None}
        best_score = -np.inf

        # Seleção de nós candidatos
        centrality = nx.degree_centrality(G)
        top_central_nodes = sorted(centrality, key=centrality.get, reverse=True)[:len(nodes)//2]
        _, _, initial_rssi_values = self.evaluate_coverage(G, [])
        weak_nodes = [node for node, rssi in zip(nodes, initial_rssi_values) 
                     if rssi < self.rssi_threshold]
        candidate_nodes = list(set(top_central_nodes) | set(weak_nodes))

        # Clusterização para posições iniciais
        positions = np.array([[x, y] for (x, y) in nodes])
        kmeans = KMeans(n_clusters=num_roteadores, n_init='auto', random_state=42)
        kmeans.fit(positions)
        centroids = [tuple(map(float, c)) for c in kmeans.cluster_centers_]
        centroid_nearest_nodes = [min(nodes, key=lambda n: np.linalg.norm(np.array(n) - np.array(c))) 
                                for c in centroids]
        candidate_nodes = list(set(candidate_nodes + centroid_nearest_nodes))

        # Otimização iterativa
        for iteration in range(max_iter):
            if len(candidate_nodes) < num_roteadores:
                candidate_nodes = nodes.copy()

            if iteration % 10 == 0:
                mutation_nodes = [nodes[i] for i in np.random.choice(
                    len(nodes), min(40, len(nodes)), replace=False)]
                candidate_nodes = list(set(candidate_nodes[:len(candidate_nodes)//2] + mutation_nodes))

            selected_indices = np.random.choice(
                len(candidate_nodes), size=num_roteadores, replace=False)
            combo = [candidate_nodes[i] for i in selected_indices]

            coverage, avg_rssi, _ = self.evaluate_coverage(G, combo)
            penalty = self.router_distance_penalty(combo)
            score = avg_rssi - 0.1 * penalty

            if score > best_score:
                best = {'coverage': coverage, 'avg_rssi': avg_rssi, 'routers': combo}
                best_score = score

                new_candidates = set()
                for router in combo:
                    neighbors = list(G.neighbors(router)) + [router]
                    new_candidates.update(neighbors)
                candidate_nodes = list(new_candidates)

        return best['routers'], best['coverage'], best['avg_rssi']

    def plot_simulation(self, G, routers, coverage, avg_rssi):
        """Visualiza a simulação com cobertura RSSI"""
        scale_factor = self.scale_factor
        pos = {n: (n[0] * scale_factor, n[1] * scale_factor) for n in G.nodes()}
        plt.figure(figsize=(16, 12))
        _, _, rssi_values = self.evaluate_coverage(G, routers)

        # Desenhar arestas
        edge_colors = [self.weight_colors.get(G[u][v].get('weight', 1), 'black') 
                      for u, v in G.edges()]
        nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=1.2, alpha=0.6)

        # Desenhar nós com cores RSSI
        nodes = nx.draw_networkx_nodes(
            G, pos, node_color=rssi_values,
            cmap='RdYlGn', vmin=-90, vmax=-30,
            node_size=80
        )

        # Destacar roteadores
        nx.draw_networkx_nodes(
            G, pos, nodelist=routers,
            node_color='black', node_size=300,
            edgecolors='yellow', linewidths=2
        )

        plt.colorbar(nodes, label='RSSI (dBm)')
        plt.title(f"Cobertura: {coverage:.1f}% | RSSI Médio: {avg_rssi:.1f} dBm")
        plt.axis('equal')
        plt.axis('off')
        plt.show()

    def run_optimization(self):
        """Executa todo o processo de otimização"""
        print("\n=== OTIMIZAÇÃO DE ROTEADORES ===")
        num_roteadores = int(input("Digite o número de roteadores: "))
        print(f"Roteador: Cisco AIR-AP-2802I-Z-K9-BR")
        print(f"Configuração: {num_roteadores} roteadores")
        print(f"TX Power: {self.tx_power} dBm, Frequência: {self.freq_mhz/1000} GHz\n")

        G = self.load_graph()
        print(f"Grafo carregado com {len(G.nodes())} nós")
        self.show_graph(G, "Grafo de Navegação Original")

        best_routers, coverage, avg_rssi = self.find_best_routers(G, num_roteadores)

        print("\n=== RESULTADOS ===")
        print(f"Melhores posições: {best_routers}")
        print(f"Cobertura: {coverage:.1f}% (acima de {self.rssi_threshold} dBm)")
        print(f"RSSI médio: {avg_rssi:.1f} dBm")

        self.plot_simulation(G, best_routers, coverage, avg_rssi)

if __name__ == "__main__":
    optimizer = RouterOptimizer()
    optimizer.run_optimization()